In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import pandas as pd
import numpy as np
import matplotlib
from IPython.display import *

import AssembleData

In [7]:

# Build Dataframe 
#MAL_df, all_genre  = AssembleData.read_files_all(verbose=True)
#MAL_df = MAL_df.drop_duplicates('id')

In [8]:
import pickle

#MAL_df = MAL_df.drop_duplicates('id')
#with open('augmented_1998_2015_data.pkl', 'w') as f:
#    pickle.dump(MAL_df, f)
#    pickle.dump(all_genre, f)

with open('augmented_1998_2015_data.pkl', 'r') as f:
    MAL_df = pickle.load(f)
    all_genre = pickle.load(f)

In [9]:
def only_genre(g):
    return MAL_df[MAL_df[g] == 1]

In [10]:
def get_std(entry):
    #print entry
    scores = []
    scores.append([10] * entry["score_10_votes"])
    scores.append([9] * entry["score_9_votes"])
    scores.append([8] * entry["score_8_votes"])
    scores.append([7] * entry["score_7_votes"])
    scores.append([6] * entry["score_6_votes"])
    scores.append([5] * entry["score_5_votes"])
    scores.append([4] * entry["score_4_votes"])
    scores.append([3] * entry["score_3_votes"])
    scores.append([2] * entry["score_2_votes"])
    scores.append([1] * entry["score_1_votes"])

    scores = [val for sublist in scores for val in sublist]

    #print scores
    return np.std(scores)

def get_df_std(df):
    scores = []
    for idx, entry in df.iterrows():
        scores.append([10] * entry["score_10_votes"])
        scores.append([9] * entry["score_9_votes"])
        scores.append([8] * entry["score_8_votes"])
        scores.append([7] * entry["score_7_votes"])
        scores.append([6] * entry["score_6_votes"])
        scores.append([5] * entry["score_5_votes"])
        scores.append([4] * entry["score_4_votes"])
        scores.append([3] * entry["score_3_votes"])
        scores.append([2] * entry["score_2_votes"])
        scores.append([1] * entry["score_1_votes"])

    scores = [val for sublist in scores for val in sublist]

    #print scores
    return np.std(scores)


In [11]:
MAL_df["std"]  = MAL_df.apply(lambda x: get_std(x), axis = 1)
MAL_df['length'] = MAL_df['duration'] * MAL_df['episodes']
MAL_df['studios'] = MAL_df['studios'].apply(lambda s: eval(s))
MAL_df['aired_start'] = pd.to_datetime(MAL_df['aired_start'])

#print MAL_df.head()

In [12]:
sig_length = MAL_df[MAL_df['length'] > 30]

stat_df = sig_length[sig_length['score_users'] > 20000]
stat_df = stat_df.reset_index()

In [13]:
#print stat_df.iloc[0]['related_ids']

#stat_df = stat_df.set_index(['id'])
#print stat_df.index

stat_df = MAL_df.sort_values('members', ascending =False)


empty_df = stat_df[stat_df['id'] == 11111111111]
covered_ids = set([])

def getRelated(entry, covered_ids, df):
    id = entry["id"]
    related_entries = []
    
    if(id not in covered_ids):
        related_entries.append(id)
        for idx in eval(df[df['id']==id].iloc[0]['related_ids']):
            if (len(df[df['id']==idx]) > 0 and id in eval(df[df['id']==idx].iloc[0]['related_ids'])):
                covered_ids.add(id)
                new_related_entries, covered_ids = getRelated(df[df['id']==idx].iloc[0], covered_ids, df)
                related_entries.extend(new_related_entries)
    return related_entries, covered_ids
        
    

for idx, entry in stat_df.iterrows():
    related_entries, covered_ids = getRelated(entry, covered_ids, stat_df)
    if(len(related_entries) != 0):
        new_entry = entry
        new_entry['related_ids'] = related_entries
        empty_df = empty_df.append(entry)
    #for idx in eval(stat_df.iloc[0]['related_ids']):
    #if (len(stat_df[stat_df['id']==idx]['title']) > 0):
    #    print stat_df[stat_df['id']==idx]['title']
        #DO MERING HERE

print len(stat_df)
print len(empty_df)
print "done"
#stat_df.iloc[0]['title']

7053
3915
done


In [140]:
combine_df = pd.DataFrame(empty_df,columns=['title', 'related_ids'])
combine_df["related_ids"] = combine_df["related_ids"].apply(lambda x: len(x))
print combine_df.sort_values('related_ids', ascending =False).head(50)

                                               title  related_ids
0                                            Pokemon           76
0                                 Sore Ike! Anpanman           44
0                                    Detective Conan           44
0                                          One Piece           42
0                                  Futari wa Precure           32
0                                   Nodame Cantabile           31
0                                    Doraemon (1979)           30
0                        Peeping Life TV: Season 1??           27
0                                          Fate/Zero           26
0                                             Naruto           24
0                                  Shakugan no Shana           22
0                                   Crayon Shin-chan           22
0                                  Kyoukai no Kanata           21
0                                     Bakemonogatari           21
0         

In [187]:
combine_df = pd.DataFrame(empty_df,columns=['id', 'title', 'related_ids'])

def averageOutSeries(entry, base_df):
    generic_df = base_df[base_df['id'] == 11111111111]
    for idx in entry['related_ids']:
        new_entry = base_df[base_df['id']==idx].iloc[0]
        generic_df = generic_df.append(new_entry)

    finalRow = base_df[base_df['id']==entry['id']].iloc[0]
    finalRow["score"] = generic_df["score"].mean()
    finalRow['related_ids'] = entry['related_ids']
    return finalRow


filter_df = combine_df["related_ids"].apply(lambda x: len(x))
#print filter_df.sort_values(ascending =False).head(200)
avg_df = combine_df[filter_df > 5].apply(lambda x: averageOutSeries(x, stat_df), axis=1)


In [188]:
series_df = pd.DataFrame(avg_df,columns=['title', 'score', 'related_ids'])
print series_df.sort_values('score', ascending =False)
#series_df = series_df[series_df["related_ids"] > 2]
#print pd.DataFrame(series_df,columns=['title', 'score']).sort_values('score', ascending =False)

                                                title     score  \
0                                            Mushishi  8.735000   
0                                      Hajime no Ippo  8.526667   
0                                            Major S1  8.407778   
0                                             Gintama  8.397500   
0                              Hunter x Hunter (2011)  8.214286   
0                                    Junjou Romantica  8.172222   
0                                  Natsume Yuujinchou  8.054286   
0                                            xxxHOLiC  8.042000   
0                                    Kuroko no Basket  8.030833   
0                                         Steins;Gate  8.014286   
0                                  Aria The Animation  8.006250   
0                                           Haikyuu!!  7.986667   
0                          Tengen Toppa Gurren Lagann  7.926667   
0                               Initial D First Stage  7.91538

# WITH GARBAGE OVA's ETC FILTERED

In [14]:
#stat_df = stat_df.set_index(['id'])
#print stat_df.index
MAL_df['norm_score'] = (MAL_df['score'] - 1.0) / 9.0
MAL_df['bayes_score'] = (MAL_df['norm_score']*MAL_df['score_users'] + 1.0) / (MAL_df['score_users'] + 2.0)

sig_length = MAL_df[((MAL_df['length'] > 30) | (MAL_df['length'] == 0))]

stat_df = sig_length
stat_df = stat_df.sort_values('members', ascending =False)

empty_df = stat_df[stat_df['id'] == 11111111111]
covered_ids = set([])

def getRelated(entry, covered_ids, df):
    id = entry["id"]
    related_entries = []
    
    if(id not in covered_ids):
        related_entries.append(id)
        for idx in eval(df[df['id']==id].iloc[0]['related_ids']):
            if (len(df[df['id']==idx]) > 0 and id in eval(df[df['id']==idx].iloc[0]['related_ids'])):
                covered_ids.add(id)
                new_related_entries, covered_ids = getRelated(df[df['id']==idx].iloc[0], covered_ids, df)
                related_entries.extend(new_related_entries)
    return related_entries, covered_ids
        
    

for idx, entry in stat_df.iterrows():
    related_entries, covered_ids = getRelated(entry, covered_ids, stat_df)
    if(len(related_entries) != 0):
        new_entry = entry
        new_entry['related_ids'] = related_entries
        empty_df = empty_df.append(entry)
    #for idx in eval(stat_df.iloc[0]['related_ids']):
    #if (len(stat_df[stat_df['id']==idx]['title']) > 0):
    #    print stat_df[stat_df['id']==idx]['title']
        #DO MERING HERE
print len(stat_df)
print len(empty_df)
print "done"
#stat_df.iloc[0]['title']

4853
3121
done


In [214]:
combine_df = pd.DataFrame(empty_df,columns=['title', 'related_ids'])

combine_df["related_ids"] = combine_df["related_ids"].apply(lambda x: len(x))
print combine_df.sort_values('related_ids', ascending =False).head(100)

                                        title  related_ids
0                                     Pokemon           39
0                           Futari wa Precure           30
0                          Sore Ike! Anpanman           29
0                                   One Piece           27
0                             Detective Conan           26
0                            Crayon Shin-chan           20
0                             Doraemon (1979)           20
0                                      Naruto           15
0                                Duel Masters           15
0                                   Fate/Zero           15
0                       Initial D First Stage           13
0                           Digimon Adventure           13
0                            Prince of Tennis           13
0                             Hidamari Sketch           12
0             Kara no Kyoukai 1: Fukan Fuukei           11
0                 Peeping Life TV: Season 1??           

In [224]:
combine_df = pd.DataFrame(empty_df,columns=['id', 'title', 'related_ids'])

def averageOutSeries(entry, base_df):
    generic_df = base_df[base_df['id'] == 11111111111]
    for idx in entry['related_ids']:
        new_entry = base_df[base_df['id']==idx].iloc[0]
        generic_df = generic_df.append(new_entry)

    finalRow = base_df[base_df['id']==entry['id']].iloc[0]
    finalRow["bayes_score"] = generic_df["bayes_score"].mean()
    finalRow['related_ids'] = entry['related_ids']
    return finalRow


filter_df = combine_df["related_ids"].apply(lambda x: len(x))
#print filter_df.sort_values(ascending =False).head(200)
avg_df = combine_df[filter_df > 3].apply(lambda x: averageOutSeries(x, stat_df), axis=1)

In [225]:
series_df = pd.DataFrame(avg_df,columns=['title', 'bayes_score', 'related_ids'])]
print series_df.sort_values('bayes_score', ascending =False)

                        title  bayes_score                     related_ids
0  Suzumiya Haruhi no Yuuutsu     0.745549  [849, 4382, 7311, 26351, 5118]
                                                title  bayes_score  \
0                                             Gintama     0.868237   
0                                            Mushishi     0.859417   
0                                  Natsume Yuujinchou     0.846655   
0                                      Hajime no Ippo     0.842872   
0                          Mahou Shoujo Madoka★Magica     0.828046   
0                                            Major S1     0.823701   
0                                    Kuroko no Basket     0.819434   
0                                      Bakemonogatari     0.812850   
0                                           Haikyuu!!     0.805423   
0                                            Bakuman.     0.802999   
0                              Hunter x Hunter (2011)     0.801573   
0         

In [227]:
avg_df = combine_df[filter_df > 1].apply(lambda x: averageOutSeries(x, stat_df), axis=1)

In [228]:
series_df = pd.DataFrame(avg_df,columns=['title', 'bayes_score', 'related_ids'])
print series_df.sort_values('bayes_score', ascending =False)

                                                title  bayes_score  \
0                                             Gintama     0.868237   
0                                            Mushishi     0.859417   
0                                  Natsume Yuujinchou     0.846655   
0                                        Cowboy Bebop     0.845551   
0                                      Hajime no Ippo     0.842872   
0                          Tengen Toppa Gurren Lagann     0.838507   
0                          Mahou Shoujo Madoka★Magica     0.828046   
0                                         Chihayafuru     0.824433   
0                                            Baccano!     0.823882   
0                                            Major S1     0.823701   
0   Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...     0.822770   
0                                             Kingdom     0.822739   
0                                    Kuroko no Basket     0.819434   
0                   